In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [3]:
candidates = session.query(Spouse).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 27766


### Write Input File

In [4]:
import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/intro/')

from tutorials.intro import load_external_labels
load_external_labels(session, Spouse, splits=[1])

AnnotatorLabels created: 2370


In [5]:
from snorkel.annotations import load_gold_labels

# L_gold = load_gold_labels(session, annotator_name='gold', split=0)
L_gold = load_gold_labels(session, annotator_name='gold', split=1)
L_gold

<2456x1 sparse matrix of type '<type 'numpy.int64'>'
	with 2370 stored elements in Compressed Sparse Row format>

In [6]:
labels = [L_gold[L_gold.get_row_index(c),0] for c in candidates]
assert(len(labels) == len(candidates))

KeyError: 174490

In [ ]:
from mturk_processing import MTurkHelper
# helper = MTurkHelper(candidates, labels, num_hits=25)
# helper = MTurkHelper(candidates, labels, pct_positive=0.5, num_hits=13)
# helper = MTurkHelper(candidates, labels, pct_positive=0.5, num_hits=12)

In [ ]:
# input_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_spouse_04_in.csv'
# helper.preprocess(input_csv_path)

### Read Output File

In [ ]:
from mturk_processing import MTurkHelper
helper = MTurkHelper()
output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_spouse_all_out.csv'
explanations = helper.postprocess(output_csv_path, candidates=candidates, verbose=True)

In [ ]:
exp_iterator = iter(explanations)

In [ ]:
exp = exp_iterator.next()
from snorkel.viewer import SentenceNgramViewer
print(exp.condition)
sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
sv

In [ ]:
for exp in explanations[:25]: print(exp.condition)

### Write Explanations File

In [ ]:
import os
from snorkel.contrib.babble import ExplanationIO

# fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/grad_explanations.tsv'
fpath = os.environ['SNORKELHOME'] + '/tutorials/babble/spouse/data/mturk_explanations_all.tsv'

expio = ExplanationIO()
expio.write(explanations, fpath)

###  Read Explanations File

In [ ]:
from pprint import pprint

explanations = expio.read(fpath)
pprint(explanations[:10])